In [60]:
import pandas as pd
import numpy as np
import requests

In [44]:
base = "https://miftaahinstitute.neolms.com/api/v2/"
endpoint = "users?$limit=10000"
# api_key = "?api_key=34549ae698ce0b382e904b7a4c79a44a9da06059e7ec4f4cea7a"
url = base + endpoint

In [45]:
resp = requests.get(url, headers={'x-api-key': '34549ae698ce0b382e904b7a4c79a44a9da06059e7ec4f4cea7a'})
json = resp.json()
df = pd.DataFrame(json.get('data'))

In [48]:
df['full_name'] = df.first_name + " " + df.last_name

In [49]:
df.shape

(1296, 71)

In [53]:
df

,id,about,active_this_month_new,administrator,allow_chat_requests,allow_friends,appear_in_searches,appear_to_online_students,archived,archived_at,...,classes_enrolled,completion_histories,groups,group_memberships,orders,affiliate_group,affiliate_orders,sessions,visits,full_name
0,11236499,None,False,True,True,True,True,True,False,None,...,{'$ref': '/api/v2/users/11236499/classes_enrol...,{'$ref': '/api/v2/users/11236499/completion_hi...,{'$ref': '/api/v2/users/11236499/groups'},{'$ref': '/api/v2/users/11236499/group_members...,{'$ref': '/api/v2/users/11236499/orders'},{'$ref': '/api/v2/users/11236499/affiliate_gro...,{'$ref': '/api/v2/users/11236499/affiliate_ord...,{'$ref': '/api/v2/users/11236499/sessions'},{'$ref': '/api/v2/users/11236499/visits'},Admin User
1,11236521,Sample instructor,False,False,True,True,True,True,False,None,...,{'$ref': '/api/v2/users/11236521/classes_enrol...,{'$ref': '/api/v2/users/11236521/completion_hi...,{'$ref': '/api/v2/users/11236521/groups'},{'$ref': '/api/v2/users/11236521/group_members...,{'$ref': '/api/v2/users/11236521/orders'},{'$ref': '/api/v2/users/11236521/affiliate_gro...,{'$ref': '/api/v2/users/11236521/affiliate_ord...,{'$ref': '/api/v2/users/11236521/sessions'},{'$ref': '/api/v2/users/11236521/visits'},Natalie Staton
2,11262456,None,False,True,True,True,True,True,False,None,...,{'$ref': '/api/v2/users/11262456/classes_enrol...,{'$ref': '/api/v2/users/11262456/completion_hi...,{'$ref': '/api/v2/users/11262456/groups'},{'$ref': '/api/v2/users/11262456/group_members...,{'$ref': '/api/v2/users/11262456/orders'},{'$ref': '/api/v2/users/11262456/affiliate_gro...,{'$ref': '/api/v2/users/11262456/affiliate_ord...,{'$ref': '/api/v2/users/11262456/sessions'},{'$ref': '/api/v2/users/11262456/visits'},Admin Miftaah
3,11303258,None,False,True,True,True,True,True,False,None,...,{'$ref': '/api/v2/users/11303258/classes_enrol...,{'$ref': '/api/v2/users/11303258/completion_hi...,{'$ref': '/api/v2/users/11303258/groups'},{'$ref': '/api/v2/users/11303258/group_members...,{'$ref': '/api/v2/users/11303258/orders'},{'$ref': '/api/v2/users/11303258/affiliate_gro...,{'$ref': '/api/v2/users/11303258/affiliate_ord...,{'$ref': '/api/v2/users/11303258/sessions'},{'$ref': '/api/v2/users/11303258/visits'},Hassan Jibril
4,11352535,None,False,False,True,True,True,True,False,None,...,{'$ref': '/api/v2/users/11352535/classes_enrol...,{'$ref': '/api/v2/users/11352535/completion_hi...,{'$ref': '/api/v2/users/11352535/groups'},{'$ref': '/api/v2/users/11352535/group_members...,{'$ref': '/api/v2/users/11352535/orders'},{'$ref': '/api/v2/users/11352535/affiliate_gro...,{'$ref': '/api/v2/users/11352535/affiliate_ord...,{'$ref': '/api/v2/users/11352535/sessions'},{'$ref': '/api/v2/users/11352535/visits'},Sondos Suleiman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,11999179,None,False,False,True,True,True,True,False,None,...,{'$ref': '/api/v2/users/11999179/classes_enrol...,{'$ref': '/api/v2/users/11999179/completion_hi...,{'$ref': '/api/v2/users/11999179/groups'},{'$ref': '/api/v2/users/11999179/group_members...,{'$ref': '/api/v2/users/11999179/orders'},{'$ref': '/api/v2/users/11999179/affiliate_gro...,{'$ref': '/api/v2/users/11999179/affiliate_ord...,{'$ref': '/api/v2/users/11999179/sessions'},{'$ref': '/api/v2/users/11999179/visits'},Mubarak Ibrahim
1292,11999392,None,False,False,True,True,True,True,False,None,...,{'$ref': '/api/v2/users/11999392/classes_enrol...,{'$ref': '/api/v2/users/11999392/completion_hi...,{'$ref': '/api/v2/users/11999392/groups'},{'$ref': '/api/v2/users/11999392/group_members...,{'$ref': '/api/v2/users/11999392/orders'},{'$ref': '/api/v2/users/11999392/affiliate_gro...,{'$ref': '/api/v2/users/11999392/affiliate_ord...,{'$ref': '/api/v2/users/11999392/sessions'},{'$ref': '/api/v2/users/11999392/visits'},Yasser Obaid
1293,11999474,None,False,False,True,True,True,True,False,None,...,{'$ref': '/api/v2/users/11999474/classes_enrol...,{'$ref': '/api/v2/use

# Connect to Google Sheets

In [1]:
import pandas as pd
import gspread
from google.oauth2 import service_account
import os

# Define the scope
scopes = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']# Add your service account file
secret_file = os.path.join(os.getcwd(), 'client_secret.json')
credentials = service_account.Credentials.from_service_account_file(secret_file, scopes=scopes)
# Authorize the clientsheet
client = gspread.authorize(credentials)
# Get the instance of the Spreadsheet
sheet = client.open('Database').worksheet("Manual Transcripts")

In [3]:
headers = ['NEO_ID',
  'Student',
  'Year',
  'Semester',
  'Class',
  'Grade',
  'Academic_Year',
  'Email',
  'Format',
  'Date_Started_Program']
table = sheet.get_all_values()
hdf = pd.DataFrame.from_records(table[1:],columns=headers)
hdf = hdf[[c for c in hdf.columns if len(c) > 0]]

In [72]:
hdf = hdf.replace('#N/A', -1)
hdf['NEO_ID'] = hdf['NEO_ID'].astype('int32')

In [4]:
hdf.to_csv("manual.csv")

In [78]:
no_neo = hdf[hdf['NEO_ID'] == -1]
no_neo['Student'].nunique()

938

## Join NEO DF and Sheets DF

In [79]:
df.dtypes

id                        int64
about                    object
active_this_month_new      bool
administrator              bool
allow_chat_requests        bool
                          ...  
affiliate_group          object
affiliate_orders         object
sessions                 object
visits                   object
full_name                object
Length: 71, dtype: object

In [80]:
hdf.dtypes

NEO_ID                   int32
Student                 object
Year                    object
Semester                object
Class                   object
Grade                   object
Academic_Year           object
Email                   object
Format                  object
Date_Started_Program    object
dtype: object

In [92]:
merged = df.merge(hdf, left_on='id', right_on='NEO_ID', how='inner')
merged

,id,about,active_this_month_new,administrator,allow_chat_requests,allow_friends,appear_in_searches,appear_to_online_students,archived,archived_at,...,NEO_ID,Student,Year,Semester,Class,Grade,Academic_Year,Email,Format,Date_Started_Program
0,11303258,None,False,True,True,True,True,True,False,None,...,11303258,Hassan Jibril,1,1,Jalalayn,CO,2021/22,hassan.jibril23@gmail.com,Advanced,September 2021
1,11303258,None,False,True,True,True,True,True,False,None,...,11303258,Hassan Jibril,1,1,Quduri,CO,2021/22,hassan.jibril23@gmail.com,Advanced,September 2021
2,11303258,None,False,True,True,True,True,True,False,None,...,11303258,Hassan Jibril,1,1,Arabic 301,CO,2021/22,hassan.jibril23@gmail.com,Advanced,September 2021
3,11303258,None,False,True,True,True,True,True,False,None,...,11303258,Hassan Jibril,2,1,Arabic,CO,2022/23,hassan.jibril23@gmail.com,Advanced,September 2021
4,11303258,None,False,True,True,True,True,True,False,None,...,11303258,Hassan Jibril,2,1,Tafsir of Surah Yusuf,CO,2022/23,hassan.jibril23@gmail.com,Advanced,September 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539,11914898,None,False,False,True,True,True,True,False,None,...,11914898,Jihan Aiyash,1,2,Arabic Grammar,A,2022/23,jihanaiyash@gmail.com,Associates,September 2022
2540,11914898,None,False,False,True,True,True,True,False,None,...,11914898,Jihan Aiyash,1,2,Prophetic Traditions,A+,2022/23,jihanaiyash@gmail.com,Associates,September 2022
2541,11914898,None,False,False,True,True,True,True,False,None,...,11914898,Jihan Aiyash,1,2,Islamic Jurisprudence,A+,2022/23,jihanaiyash@gmail.com,Associates,September 2022
2542,11914898,None,False,False,True,True,True,True,False,None,...,11914898,Jihan Aiyash,1,2,Prophetic Biography,B,2022/23,jihanaiyash@gmail.com,Associates,September 2022


In [90]:
mismatched = merged[merged['full_name'] != merged['Student']]
mismatched[['id', 'full_name', 'Student']].drop_duplicates()

,id,full_name,Student
52,11362821,Mahmuda Ali,Mahmuda Bilkis Ali
70,11362839,Saira Ali,Saira Choudhri
77,11362846,shah sajjad,Shah Sajjad
94,11362850,Safia Syeda,Safia Syed
114,11362867,Armaan Mohajir,Armaan H Mohajir
178,11363011,Humaira AbdulRehman,Humaira Abdul Rehman
189,11363014,maham irfan,Maham Irfan
267,11363160,Hassan Amin,Hassan H Amin
270,11363182,Aisha Khan,Arshi Wasiuddin
293,11363337,Abdul Amer,Abdul K Amer
